In [1]:
import polars as pl

In [2]:
%%time
cats = pl.read_csv("category_tree.csv"
           ).with_columns(pl.when(pl.col("parent_id").is_null()).then(pl.lit(0)).otherwise(-99).cast(pl.Int8).alias("level"),
                          pl.col("parent_id").cast( pl.Int16),
                          pl.col("cat_id").cast( pl.Int16),
                          #pl.lit(0).cast( pl.Int8).alias("end"),
                          #pl.lit(0).cast( pl.Int8).alias("population_cats"),
                          #pl.lit(0).cast( pl.Int8).alias("population_names"),
                         )#.with_columns( pl.when(pl.col("parent_id").is_null()).then(pl.lit(0)).otherwise(-1) )
cats = cats.join(cats[["cat_id","level"]],left_on="parent_id",right_on="cat_id",how="left")
cats = cats.with_columns( pl.when(pl.col("level")>=0).then(pl.col("level")).otherwise(pl.col("level_right")+1) ).drop("level_right")
cats = cats.join(cats[["cat_id","level"]],left_on="parent_id",right_on="cat_id",how="left")
cats = cats.with_columns( pl.when(pl.col("level")>=0).then(pl.col("level")).otherwise(pl.col("level_right")+1) ).drop("level_right")
cats = cats.join(cats[["cat_id","level"]],left_on="parent_id",right_on="cat_id",how="left")
cats = cats.with_columns( pl.when(pl.col("level")>=0).then(pl.col("level")).otherwise(pl.col("level_right")+1) ).drop("level_right")
cats = cats.join(cats[["cat_id","level"]],left_on="parent_id",right_on="cat_id",how="left")
cats = cats.with_columns( pl.when(pl.col("level")>=0).then(pl.col("level")).otherwise(pl.col("level_right")+1) ).drop("level_right")

cats = cats.with_columns( pl.when(pl.col("level")==4).then(pl.col("cat_id")).alias("p4") )

cats = cats.with_columns( pl.when(pl.col("level")==4).then(pl.col("parent_id")).alias("p3") )
cats = cats.with_columns( pl.when(pl.col("level")==3).then(pl.col("cat_id")).otherwise(pl.col("p3")).alias("p3") )

cats = cats.join(cats[["cat_id","parent_id"]],left_on="p3",right_on="cat_id",how="left").rename({"parent_id_right":"p2"})
cats = cats.with_columns( pl.when(pl.col("level")==2).then(pl.col("cat_id")).otherwise(pl.col("p2")).alias("p2") )

cats = cats.join(cats[["cat_id","parent_id"]],left_on="p2",right_on="cat_id",how="left").rename({"parent_id_right":"p1"})
cats = cats.with_columns( pl.when(pl.col("level")==1).then(pl.col("cat_id")).otherwise(pl.col("p1")).alias("p1") )

cats = cats.join(cats[["cat_id","parent_id"]],left_on="p1",right_on="cat_id",how="left").rename({"parent_id_right":"p0"})
cats = cats.with_columns( pl.when(pl.col("level")==0).then(pl.col("cat_id")).otherwise(pl.col("p0")).alias("p0") )

cats

CPU times: user 63.3 ms, sys: 65.3 ms, total: 129 ms
Wall time: 176 ms


cat_id,parent_id,cat_name,level,p4,p3,p2,p1,p0
i16,i16,str,i8,i16,i16,i16,i16,i16
1,null,"""Электроника""",0,null,null,null,null,1
2,null,"""Ноутбуки и компьютеры""",0,null,null,null,null,2
3,null,"""Бытовая техника""",0,null,null,null,null,3
4,null,"""Детские товары""",0,null,null,null,null,4
5,null,"""Дача, сад и огород""",0,null,null,null,null,5
…,…,…,…,…,…,…,…,…
31426,14355,"""Аккумуляторы для мобильных тел…",4,31426,14355,1005,101,1
31431,14355,"""Коннекторы и контейнеры для те…",4,31431,14355,1005,101,1
31433,14355,"""Корпусные детали для телефонов""",4,31433,14355,1005,101,1


In [3]:
#population_cats	end	population_trains

#cats = 
cats0 = pl.read_csv("category_tree.csv"
           ).with_columns(pl.when(pl.col("parent_id").is_null()).then(pl.lit(0)).otherwise(-99).cast(pl.Int8).alias("level"),
                          pl.col("parent_id").cast( pl.Int16),
                          pl.col("cat_id").cast( pl.Int16),
                          pl.lit(0).cast( pl.Int8).alias("end"),
                          #pl.lit(0).cast( pl.Int8).alias("population_cats"),
                          #pl.lit(0).cast( pl.Int8).alias("population_names"),
                         )#.with_columns( pl.when(pl.col("parent_id").is_null()).then(pl.lit(0)).otherwise(-1) )
cats0=cats0.join(cats[["cat_id","parent_id"]],left_on="cat_id",right_on="parent_id",how="left"
         ).rename({"cat_id_right":"population_cats"}
                 ).group_by("cat_id"
                           ).agg(pl.col("population_cats").count().cast(pl.Int8)
                                ).with_columns(pl.when(pl.col("population_cats")==0
                                                      ).then(pl.lit(1)
                                                            ).otherwise(pl.lit(0)
                                                                       ).cast( pl.Int8).alias("end")
                                              )
cats0
cats1=cats0.select("cat_id"
                  ).join(pl.read_parquet("labeled_train.parquet"
                                ).group_by("cat_id").len(
                                        ).with_columns(pl.col("cat_id").cast(pl.Int16)),
                 on="cat_id",
                 how="left"
                ).rename({"len":"population_trains"}
                        ).fill_null(0)


cats=cats.join(cats0,on="cat_id",how="left").join(cats1,on="cat_id",how="left")
cats

cat_id,parent_id,cat_name,level,p4,p3,p2,p1,p0,population_cats,end,population_trains
i16,i16,str,i8,i16,i16,i16,i16,i16,i8,i8,u32
1,null,"""Электроника""",0,null,null,null,null,1,18,0,540
2,null,"""Ноутбуки и компьютеры""",0,null,null,null,null,2,15,0,20
3,null,"""Бытовая техника""",0,null,null,null,null,3,7,0,0
4,null,"""Детские товары""",0,null,null,null,null,4,11,0,0
5,null,"""Дача, сад и огород""",0,null,null,null,null,5,9,0,0
…,…,…,…,…,…,…,…,…,…,…,…
31426,14355,"""Аккумуляторы для мобильных тел…",4,31426,14355,1005,101,1,0,1,0
31431,14355,"""Коннекторы и контейнеры для те…",4,31431,14355,1005,101,1,0,1,0
31433,14355,"""Корпусные детали для телефонов""",4,31433,14355,1005,101,1,0,1,0


In [4]:
#population_trains_full
labeled=pl.read_parquet("labeled_train.parquet"
               ).with_columns(pl.col("cat_id").cast(pl.Int16))
ps=[]                              
for level in range(5):                            
    ps.append(labeled.join(cats,on="cat_id",how="left"
                          ).group_by(f"p{level}").len().rename({f"p{level}":"cat_id","len":"population_trains_full"})#.sort("p0")
             )
cats=cats.join( cats.select("cat_id").join(pl.concat(ps),on="cat_id",how="left").fill_null(0),
          on="cat_id",how="left")
cats

cat_id,parent_id,cat_name,level,p4,p3,p2,p1,p0,population_cats,end,population_trains,population_trains_full
i16,i16,str,i8,i16,i16,i16,i16,i16,i8,i8,u32,u32
1,null,"""Электроника""",0,null,null,null,null,1,18,0,540,307950
2,null,"""Ноутбуки и компьютеры""",0,null,null,null,null,2,15,0,20,180160
3,null,"""Бытовая техника""",0,null,null,null,null,3,7,0,0,145342
4,null,"""Детские товары""",0,null,null,null,null,4,11,0,0,6583
5,null,"""Дача, сад и огород""",0,null,null,null,null,5,9,0,0,805
…,…,…,…,…,…,…,…,…,…,…,…,…
31426,14355,"""Аккумуляторы для мобильных тел…",4,31426,14355,1005,101,1,0,1,0,0
31431,14355,"""Коннекторы и контейнеры для те…",4,31431,14355,1005,101,1,0,1,0,0
31433,14355,"""Корпусные детали для телефонов""",4,31433,14355,1005,101,1,0,1,0,0


In [8]:
cats.filter(pl.col("end")==1).filter(pl.col("level")==1).filter(pl.col("population_trains")<=3)

cat_id,parent_id,cat_name,level,p4,p3,p2,p1,p0,population_cats,end,population_trains,population_trains_full
i16,i16,str,i8,i16,i16,i16,i16,i16,i8,i8,u32,u32
117,1,"""Радиоуправляемые модели""",1,null,null,null,117,1,0,1,0,0
152,4,"""Детская одежда и обувь""",1,null,null,null,152,4,0,1,0,0
164,5,"""Парники и теплицы""",1,null,null,null,164,5,0,1,0,0
169,6,"""Макароны, крупы""",1,null,null,null,169,6,0,1,0,0
170,6,"""Растительное масло""",1,null,null,null,170,6,0,1,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…
417,22,"""Школьная форма для мальчиков""",1,null,null,null,417,22,0,1,0,0
438,28,"""Игровые ноутбуки""",1,null,null,null,438,28,0,1,0,0
445,28,"""4K гейминг""",1,null,null,null,445,28,0,1,0,0


In [10]:
%%time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_yandexgpt_8b"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    #device_map="auto"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_length": 1024000,
  "max_new_tokens": 2048,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.6,
  "top_p": 0.95
}

CPU times: user 12.8 s, sys: 5.99 s, total: 18.8 s
Wall time: 11.9 s


In [15]:
%%time
for name in cats.filter(pl.col("end")==1
                       ).filter(pl.col("level")==1
                               ).filter(pl.col("population_trains")<=3
                                       )["cat_name"].to_list():
    print(name)
    query = f"Составь список товаров категории '{name}'."
    prompt = tokenizer.apply_chat_template([{
        "role": "user",
        "content": query
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    data.pop("token_type_ids", None)
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    print(query)
    print(output)
    print()
    print("==============================")
    print()    

Радиоуправляемые модели
Составь список товаров категории 'Радиоуправляемые модели'.
1. Радиоуправляемые автомобили (гоночные, багги, раллийные)
2. Радиоуправляемые мотоциклы и скутеры
3. Радиоуправляемые вертолёты (классические, коптеры)
4. Радиоуправляемые самолёты (пилотажные, гоночные, фристайл)
5. Радиоуправляемые квадрокоптеры и мультикоптеры
6. Радиоуправляемая авиация (дирижабли, планеры, парапланы)
7. Радиоуправляемые лодки (морские, речные, подводные)
8. Радиоуправляемые поезда
9. Радиоуправляемые танки и бронетранспортёры
10. Гибридные радиоуправляемые модели (например, автомобиль-квадроцикл или самолёт с двигателем внутреннего сгорания)
11. Аксессуары для моделей: двигатели, пропеллеры, винты, аккумуляторные батареи, зарядные устройства, контроллеры управления, антенны
12. Запасные части и инструменты для обслуживания моделей
13. Оборудование для сборки и модификации моделей
14. Книги по сборке и управлению моделями
15. Онлайн курсы по пилотированию моделей
16. Специализиров

Составь список товаров категории 'Орехи, семена, сухофрукты'.
1. Арахис жареный и нежареный
2. Грецкие орехи очищенные и неочищенные
3. Миндаль (очищенный)
4. Фисташки (в скорлупе или без)
5. Кешью (целые или дробленые)
6. Фундук (очищенный)
7. Кедровые орешки
8. Семена подсолнечника жареные и сырые
9. Тыквенные семечки очищенные и необработанные
10. Лен молотый/цельный
11. Кунжут белый и черный
12. Чиа семена
13. Макадамия орех
14. Бразильский орех
15. Сушеные фрукты: изюм, курага, чернослив, инжир, финики, манго сушеное
16. Ореховая смесь ассорти
17. Мёд с орехами и семенами
18. Шоколадно-ореховые плитки
19. Козинаки на основе орехов и семян
20. Готовые смеси для выпечки с добавлением орехов и семечек

Этот список включает наиболее популярные товары данной категории, однако ассортимент может варьироваться в зависимости от магазина или производителя.


Хлеб и выпечка
Составь список товаров категории 'Хлеб и выпечка'.
1. Хлеб пшеничный
2. Хлеб ржаной
3. Багет
4. Булочки для бургеров
5.

Составь список товаров категории 'Спортивные дисциплины'.
Вот пример списка товаров для различных спортивных дисциплин:

1. Футбол:
- Мячи футбольные
- Форма футбольная (футболки, шорты, гетры)
- Бутсы
- Тренировочные щиты и конусы
- Ворота футбольные
- Насосы для мячей
- Футбольные сумки

2. Баскетбол:
- Мячи баскетбольные
- Спортивная форма (майки, шорты, носки)
- Сетка для баскетбольной корзины
- Стойки для баскетбольных корзин
- Напольное покрытие для зала
- Баскетбольные стойки
- Сумки для баскетбола

3. Волейбол:
- Мячи волейбольные
- Спортивная одежда (комбинезоны, футболки, шорты)
- Сетки для волейболных сеток
- Антенны для волейбольных сеток
- Пляжные сетки для волейбола
- Таблицы с результатами матчей
- Сумки для волейбола

4. Теннис:
- Ракетки теннисные
- Струны для ракетки
- Мячи теннисные
- Чехлы для ракеток
- Перчатки для игры в теннис
- Сумки для тенниса
- Матрацы для настольного тенниса
- Шарики для пинг-понга

5. Плавание:
- Купальники женские/мужские
- Плавки мужские


Составь список товаров категории 'Надувная, плетеная и прочая мебель'.
1. Надувной матрас (с насосом)
2. Надувные кровати
3. Надувные диваны
4. Пляжные шезлонги надувные
5. Круглые надувные кресла
6. Надувные пуфы
7. Плетеные стулья
8. Плетенные столы
9. Подвесные кресла из ротанга или лозы
10. Гамаки (традиционные и современные варианты)
11. Качели-лодки
12. Детские игровые домики надувные
13. Палатки для отдыха на природе с надувными элементами
14. Складные стулья из ткани или пластика
15. Табуреты складные
16. Раскладные столики
17. Тенты от солнца и дождя
18. Подушки декоративные плетёные
19. Шезлонги плетеные
20. Коврики пляжные плетеные
21. Мебель для сада и террасы: скамьи, тумбы, полки для растений
22. Стулья для веранды и беседок
23. Барные стойки пластиковые и металлические с мягкими сиденьями
24. Модульные системы мебели для открытых пространств
25. Занавесы и перегородки тканевые для разделения пространства
26. Садовые качели различных конструкций
27. Подставки под цветочны

Составь список товаров категории 'Чертежные инструменты'.
1. Линейки чертежные: металлические, пластиковые, деревянные.
2. Угольники: прямые и угольные (90 градусов), различные размеры.
3. Циркули: обычные, вращающиеся, с разными типами ножек.
4. Транспортиры: механические и электронные, для измерения углов в градусах.
5. Лекала: кривые, прямые, изогнутые, используемые для создания кривых линий на чертежах.
6. Рейсшины: ручные и механизированные, для проведения параллельных прямых линий.
7. Штангенциркули: для точных измерений расстояний между точками.
8. Микрокалькуляторы: инженерные модели для выполнения сложных расчетов при проектировании.
9. Чертежные доски: большие листы или специальные панели для размещения чертежей.
10. Карандаши: графитные различной твердости (H, B) для рисования тонких линий.
11. Маркеры: водорастворимые, спиртовые, перманентные разных цветов для выделения элементов чертежа.
12. Ластик: мягкий или твердый, для коррекции ошибок на бумаге или кальке.
13. Кальки: